In [1]:
import requests
from bs4 import BeautifulSoup
import os
import datetime
import time
import re

headers= {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36",
    "referer": "https://www.ptt.cc/bbs/NSwitch/index2007.html"
}

In [ ]:
url = "https://www.ptt.cc/bbs/movie/index{}.html"  #9389

page = 9389
while True:
    print(page ,"頁")
    
    res = requests.get(url.format(page) , headers=headers)
    if res.status_code == 500:
        break
        
    soup = BeautifulSoup(res.text, "html.parser")
    soup_list = soup.select("div[class='title']")
    for i in soup_list:
        title = i.text
        title = re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])","",title)
        if i.select("a") == []:
            pass
        else:
            try:
                link = "https://www.ptt.cc" + i.select("a")[0]["href"]

                article_res = requests.get(link, headers=headers)
                article_soup = BeautifulSoup(article_res.text, "html.parser")
                article = article_soup.select("div[id='main-content']")[0].text.split("※ 發信站")[0]
                tmp_date = article_soup.select("span[class='article-meta-value']")[3].text
                date = datetime.datetime.strptime(tmp_date, "%a %b %d %H:%M:%S %Y")
                
            except IndexError:
                pass
            

            print(title)
            print(date)
            print(link)
            print(article)
            
            if not os.path.exists("./ptt_movies"):
                os.mkdir("./ptt_movies")
            if not os.path.exists("./ptt_movies/{}".format(str(date)[:7])):
                os.mkdir("./ptt_movies/{}".format(str(date)[:7]))
            with open("./ptt_movies/{}/{}.txt".format(str(date)[:7], title), "w", encoding="utf-8") as f:
                f.write(title + "\n" + str(date) + "\n" + link + "\n" + article)
    
    
    if date < datetime.datetime(2020,1,1):
        break        
    
    page += 1